In [ ]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent, MultipleObjectsError


import ipywidgets as ipw
from IPython.display import display, clear_output

# Install and/or update plugins

In [ ]:
def on_setup_plugins(b):
    with setup_plugin_output:
        clear_output()
        ! pip install -e ./plugins/
        ! reentry scan -r aiida

setup_plugin_output = ipw.Output()
btn_setup_plugins = ipw.Button(description="Install stm plugins")
btn_setup_plugins.on_click(on_setup_plugins)
display(btn_setup_plugins, setup_plugin_output)

In [ ]:
def on_list_plugins(b):
    with list_plugin_output:
        ! verdi calculation plugins

list_plugin_output = ipw.Output()
btn_list_plugins = ipw.Button(description="List all plugins")
btn_list_plugins.on_click(on_list_plugins)
display(btn_list_plugins, list_plugin_output)

# Setup STM codes

In [ ]:
## -------------------------------------------------------
## List computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True})
all_computers = qb.all()

computer_selection = [comp[0].get_name() for comp in all_computers]

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer",
                             options=['']+computer_selection)

def show_codes(c):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True}, project=['name'], tag='computer')
    qb.append(Code, project=['*'], has_computer='computer')
    with code_output:
        clear_output()
        print("Codes on " + drop_computer.value + ":")
        for e in qb.all():
            if e[0] == drop_computer.value:
                print(e[1])
                
code_output = ipw.Output()

drop_computer.observe(show_codes, names='value')

show_codes(0)

display(ipw.VBox([drop_computer, code_output]))

Set the remote path where to install codes. Needs to be a location available to the calculation nodes (e.g. not `/project/` on Piz-Daint). 

Example: `/users/username/aiida-soft`

In [ ]:
inp_username = ipw.Text(description="Install path:", layout=layout, style=style)
display(inp_username)

Retrieve scripts to the install path at remote machine

In [ ]:
def retrive_scripts(b):
    
    global version_id, code_dir_path
    
    commit_hash = ! git ls-remote "https://github.com/eimrek/atomistic_tools.git"
    version_id = commit_hash[0][:6]

    code_dir_name = "at_"+version_id

    code_remote_path = inp_username.value

    code_dir_path = code_remote_path+"/"+code_dir_name

    computer = Computer.get(drop_computer.value)

    with retrieve_output:
        
        clear_output()

        # ssh key needs to be set
        ! ssh {computer.hostname} "wget https://github.com/eimrek/atomistic_tools/archive/master.zip -O {code_remote_path}/{code_dir_name}.zip"
        ! ssh {computer.hostname} "cd {code_remote_path} && unzip -o {code_dir_name}.zip && rm -rf {code_dir_name} && mv atomistic_tools-master {code_dir_name}"
        ! ssh {computer.hostname} "cd {code_remote_path}/{code_dir_name} && chmod u+x evaluate_morbs_in_box_mpi.py stm_sts_extrap_from_npz.py"

code_dir_path = None
version_id = None

retrieve_output = ipw.Output()

retrieve_button = ipw.Button(description="Retrieve scripts")

retrieve_button.on_click(retrive_scripts)

display(retrieve_button, retrieve_output)


## Set up the remote codes in Aiida

For orbital evaluation script, python instance with MPI and ASE is needed.

Please provide the correct bash commands to make this available (this text will be prepended to each slurm script).

In [ ]:
eval_prepend_text = ipw.Textarea(description="Prepend text:",
             layout = {'width': '70%'},
             value =
                "module load cray-python/3.6.1.1\n" +
                "export PYTHONPATH=$PYTHONPATH:\"/users/keimre/soft/ase\"\n"
            )

display(eval_prepend_text)

In [ ]:
def setup_eval_mol_orb():
    
    # Eval mol. orbitals
    # Python with MPI support and ASE
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += eval_prepend_text.value
    prepend_text += "### code prepend_text end ###\n"

    code_label = "py_eval_morbs_"+version_id

    code_full_name = code_label + "@" + drop_computer.value

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(Computer.get(drop_computer.value), code_dir_path + "/evaluate_morbs_in_box_mpi.py"))
        code.label = code_label
        code.description = "Python code to evaluate mol. orbitals on a grid"
        code.set_input_plugin_name("stm.eval_morbs")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_eval_setup_click(b):
    with eval_setup_out:
        clear_output()
        setup_eval_mol_orb()
    
eval_setup_out = ipw.Output()

eval_setup_btn = ipw.Button(description="Setup eval morbs")
eval_setup_btn.on_click(on_eval_setup_click)

display(eval_setup_btn, eval_setup_out)

For image creation script, python instance with matplotlib is needed. (In addition to the usual ase, numpy, ...)

Please provide the correct bash commands to make this available (this text will be prepended to each slurm script).

In [ ]:
imag_prepend_text = ipw.Textarea(description="Prepend text:",
             layout = {'width': '70%'},
             value =
                "export PATH='/users/keimre/soft/miniconda3/bin':$PATH\n"
            )

display(imag_prepend_text)

In [ ]:
def setup_stm_imag():
    
    # Eval mol. orbitals
    # Python with MPI support and ASE
    prepend_text  = "### code prepend_text start ###\n"
    prepend_text += imag_prepend_text.value
    prepend_text += "mkdir stm_output\n"
    prepend_text += "### code prepend_text end ###\n"

    code_label = "py_stm_sts_"+version_id

    code_full_name = code_label + "@" + drop_computer.value

    try:
        Code.get_from_string(code_full_name)
    except (NotExistent, MultipleObjectsError):            
        code = Code(remote_computer_exec=(Computer.get(drop_computer.value), code_dir_path + "/evaluate_morbs_in_box_mpi.py"))
        code.label = code_label
        code.description = "Python code to calculate STM and STS from the output of py_eval_morbs"
        code.set_input_plugin_name("stm.stm_image")
        code.set_prepend_text(prepend_text)
        code.set_append_text("")
        code.store()
        ! verdi code show "{code_full_name}"
    else:
        print("Code '{}' already existent".format(code_full_name))
        
def on_imag_setup_click(b):
    with imag_setup_out:
        clear_output()
        setup_stm_imag()
    
imag_setup_out = ipw.Output()

imag_setup_btn = ipw.Button(description="Setup stm imag")
imag_setup_btn.on_click(on_imag_setup_click)

display(imag_setup_btn, imag_setup_out)